# 加州房价预测：端到端机器学习项目

本项目演示完整的机器学习工作流程，包括：
- 数据获取与加载
- 探索性数据分析 (EDA)
- 数据预处理与特征工程
- 模型训练与评估
- 超参数调优
- 模型部署准备

数据来源：1990年加州人口普查数据

## 1. 环境配置与数据获取

In [ ]:
# 标准库
import os
import tarfile
import urllib.request
from pathlib import Path

# 数据处理
import numpy as np
import pandas as pd

# 可视化
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# 机器学习
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy import stats
import joblib

# 配置
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)

In [ ]:
# 数据源配置
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = Path("datasets/housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"


def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """
    下载并解压加州房价数据集。
    
    Parameters
    ----------
    housing_url : str
        数据集下载地址
    housing_path : Path
        本地存储路径
    """
    housing_path.mkdir(parents=True, exist_ok=True)
    tgz_path = housing_path / "housing.tgz"
    
    if not (housing_path / "housing.csv").exists():
        print(f"正在下载数据集: {housing_url}")
        urllib.request.urlretrieve(housing_url, tgz_path)
        with tarfile.open(tgz_path) as housing_tgz:
            housing_tgz.extractall(path=housing_path)
        print("下载完成")
    else:
        print("数据集已存在，跳过下载")


fetch_housing_data()

## 2. 数据加载与初步观察

In [ ]:
def load_housing_data(housing_path=HOUSING_PATH):
    """
    加载加州房价数据集。
    
    Returns
    -------
    pd.DataFrame
        包含20640条记录的房价数据
    """
    csv_path = housing_path / "housing.csv"
    return pd.read_csv(csv_path)


housing = load_housing_data()
print(f"数据集形状: {housing.shape}")
housing.head()

In [ ]:
# 查看数据类型和缺失值情况
# total_bedrooms 列存在缺失值，需要在预处理阶段填充
housing.info()

In [ ]:
# ocean_proximity 是分类特征，查看其取值分布
# 数据存在类别不平衡：ISLAND类别样本极少(5条)
housing['ocean_proximity'].value_counts()

In [ ]:
# 数值特征统计摘要
# 注意：median_house_value 最大值500001可能是截断值
housing.describe()

In [ ]:
# 数据分布直方图
# 观察要点：
# 1. median_income 已被缩放（单位：万美元）
# 2. housing_median_age 和 median_house_value 存在截断
# 3. 多数特征呈右偏分布，可能需要对数变换
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
housing.hist(bins=50, ax=axes.flatten()[:9])
plt.tight_layout()
plt.show()

## 3. 数据集划分

采用分层抽样确保训练集和测试集具有相似的收入分布，因为收入是预测房价的关键特征。

In [ ]:
def split_train_test(data, test_ratio, seed=42):
    """
    手动实现训练集/测试集划分（教学演示用）。
    
    Parameters
    ----------
    data : pd.DataFrame
        原始数据集
    test_ratio : float
        测试集比例
    seed : int
        随机种子，保证可复现性
    
    Returns
    -------
    tuple
        (训练集, 测试集)
    """
    np.random.seed(seed)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]


# 演示：简单随机划分
train_set_demo, test_set_demo = split_train_test(housing, 0.2)
print(f"简单划分: 训练集 {len(train_set_demo)}, 测试集 {len(test_set_demo)}")

In [ ]:
from zlib import crc32


def test_set_check(identifier, test_ratio):
    """
    基于哈希值判断样本是否属于测试集。
    使用哈希确保数据更新后划分保持一致。
    """
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32


def split_train_test_by_id(data, test_ratio, id_column):
    """
    基于标识符的稳定数据集划分。
    当数据集更新时，同一样本始终归属同一集合。
    """
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]


# 使用经纬度组合作为稳定标识符
housing_with_id = housing.copy()
housing_with_id['id'] = housing['longitude'] * 1000 + housing['latitude']
train_set_hash, test_set_hash = split_train_test_by_id(housing_with_id, 0.2, 'id')
print(f"哈希划分: 训练集 {len(train_set_hash)}, 测试集 {len(test_set_hash)}")

In [ ]:
# 创建收入分层类别
# 将连续收入值离散化为5个层次，便于分层抽样
housing['income_cat'] = pd.cut(
    housing['median_income'],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5]
)

fig, ax = plt.subplots(figsize=(8, 5))
housing['income_cat'].hist(ax=ax)
ax.set_xlabel('收入类别')
ax.set_ylabel('样本数量')
ax.set_title('收入分层分布')
plt.show()

In [ ]:
# 分层抽样：确保各收入层次在训练集和测试集中比例一致
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index].copy()
    strat_test_set = housing.loc[test_index].copy()

# 验证分层效果
print("各收入层次占比:")
print(strat_train_set['income_cat'].value_counts(normalize=True).sort_index())

# 移除辅助列
for dataset in (strat_train_set, strat_test_set):
    dataset.drop('income_cat', axis=1, inplace=True)

# 同步移除原始数据中的辅助列
housing.drop('income_cat', axis=1, inplace=True)

print(f"\n最终划分: 训练集 {len(strat_train_set)}, 测试集 {len(strat_test_set)}")

## 4. 探索性数据分析 (EDA)

In [ ]:
# 在训练集副本上进行探索，避免数据泄露
housing_eda = strat_train_set.copy()

In [ ]:
# 地理分布可视化
# alpha=0.1 使重叠区域更明显，揭示人口密集区
fig, ax = plt.subplots(figsize=(10, 7))
housing_eda.plot(
    kind='scatter', x='longitude', y='latitude',
    alpha=0.1, ax=ax
)
ax.set_title('加州房屋地理分布')
plt.show()

In [ ]:
# 多维度地理可视化
# 圆圈大小：人口密度
# 颜色深浅：房价中位数
fig, ax = plt.subplots(figsize=(12, 8))
scatter = housing_eda.plot(
    kind='scatter', x='longitude', y='latitude',
    alpha=0.4,
    s=housing_eda['population'] / 100,
    c='median_house_value',
    cmap='jet',
    colorbar=True,
    label='人口',
    ax=ax
)
ax.set_title('加州房价地理热力图')
ax.legend()
plt.show()

In [ ]:
# 相关性分析
# 注意：皮尔逊相关系数只能捕捉线性关系
housing_num = housing_eda.select_dtypes(include=[np.number])
corr_matrix = housing_num.corr()

print("与房价中位数的相关性（按绝对值排序）:")
print(corr_matrix['median_house_value'].sort_values(key=abs, ascending=False))

In [ ]:
# 关键特征散点矩阵
# 对角线为直方图，其他位置为两两散点图
attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
fig = scatter_matrix(housing_eda[attributes], figsize=(12, 8))
plt.suptitle('关键特征散点矩阵', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 收入与房价关系详细分析
# 可见强正相关，但存在水平条纹（数据截断）
fig, ax = plt.subplots(figsize=(10, 6))
housing_eda.plot(
    kind='scatter', x='median_income', y='median_house_value',
    alpha=0.1, ax=ax
)
ax.set_xlabel('收入中位数 (万美元)')
ax.set_ylabel('房价中位数 (美元)')
ax.set_title('收入与房价关系')
plt.show()

In [ ]:
# 特征工程探索：组合特征可能更具预测力
housing_eda['rooms_per_household'] = housing_eda['total_rooms'] / housing_eda['households']
housing_eda['bedrooms_per_room'] = housing_eda['total_bedrooms'] / housing_eda['total_rooms']
housing_eda['population_per_household'] = housing_eda['population'] / housing_eda['households']

# 重新计算相关性
housing_num_ext = housing_eda.select_dtypes(include=[np.number])
corr_matrix_ext = housing_num_ext.corr()

print("新增特征后与房价的相关性:")
print(corr_matrix_ext['median_house_value'].sort_values(key=abs, ascending=False))

## 5. 数据预处理

构建可复用的数据处理流水线。

In [ ]:
# 分离特征和标签
housing_features = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

In [ ]:
# 缺失值处理演示
# 使用中位数填充 total_bedrooms 列的缺失值
imputer = SimpleImputer(strategy='median')

housing_num = housing_features.select_dtypes(include=[np.number])
imputer.fit(housing_num)

print("各特征中位数（用于填充缺失值）:")
print(dict(zip(housing_num.columns, imputer.statistics_)))

In [ ]:
# 分类特征编码演示
housing_cat = housing_features[['ocean_proximity']]

# 序数编码（有序分类时使用）
ordinal_encoder = OrdinalEncoder()
housing_cat_ordinal = ordinal_encoder.fit_transform(housing_cat)
print("序数编码类别:", ordinal_encoder.categories_)

# 独热编码（无序分类时使用）
onehot_encoder = OneHotEncoder(sparse_output=False)
housing_cat_onehot = onehot_encoder.fit_transform(housing_cat)
print(f"\n独热编码形状: {housing_cat_onehot.shape}")
print("独热编码特征:", onehot_encoder.get_feature_names_out())

In [ ]:
# 自定义转换器：添加组合特征
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    """
    特征工程转换器：生成房屋相关的组合特征。
    
    Parameters
    ----------
    add_bedrooms_per_room : bool
        是否添加 bedrooms_per_room 特征
    """
    
    # 列索引（基于数值特征顺序）
    ROOMS_IX = 3
    BEDROOMS_IX = 4
    POPULATION_IX = 5
    HOUSEHOLDS_IX = 6
    
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, self.ROOMS_IX] / X[:, self.HOUSEHOLDS_IX]
        population_per_household = X[:, self.POPULATION_IX] / X[:, self.HOUSEHOLDS_IX]
        
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, self.BEDROOMS_IX] / X[:, self.ROOMS_IX]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]


# 测试自定义转换器
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=True)
housing_extra_attribs = attr_adder.fit_transform(housing_num.values)
print(f"添加组合特征后形状: {housing_extra_attribs.shape}")

In [ ]:
# 数值特征处理流水线
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),      # 缺失值填充
    ('attribs_adder', CombinedAttributesAdder()),       # 特征工程
    ('std_scaler', StandardScaler()),                   # 标准化
])

# 完整数据处理流水线
num_attribs = list(housing_num.columns)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs),
])

# 处理训练数据
housing_prepared = full_pipeline.fit_transform(housing_features)
print(f"处理后数据形状: {housing_prepared.shape}")

## 6. 模型训练与评估

In [ ]:
# 基准模型：线性回归
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(f"线性回归 - 训练集 RMSE: ${lin_rmse:,.2f}")

In [ ]:
# 决策树回归
# RMSE=0 表明严重过拟合
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(f"决策树 - 训练集 RMSE: ${tree_rmse:,.2f} (过拟合警告)")

In [ ]:
def display_scores(scores, model_name):
    """显示交叉验证结果。"""
    print(f"\n{model_name} - 10折交叉验证结果:")
    print(f"  RMSE 均值: ${scores.mean():,.2f}")
    print(f"  RMSE 标准差: ${scores.std():,.2f}")
    print(f"  各折 RMSE: {[f'${s:,.0f}' for s in scores]}")


# 决策树交叉验证
tree_scores = cross_val_score(
    tree_reg, housing_prepared, housing_labels,
    scoring='neg_mean_squared_error', cv=10
)
tree_rmse_scores = np.sqrt(-tree_scores)
display_scores(tree_rmse_scores, "决策树")

# 线性回归交叉验证
lin_scores = cross_val_score(
    lin_reg, housing_prepared, housing_labels,
    scoring='neg_mean_squared_error', cv=10
)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores, "线性回归")

In [ ]:
# 随机森林回归
# 使用简化参数进行快速测试
forest_reg = RandomForestRegressor(n_estimators=30, random_state=42, n_jobs=-1)
forest_reg.fit(housing_prepared, housing_labels)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
print(f"随机森林 - 训练集 RMSE: ${forest_rmse:,.2f}")

# 交叉验证
forest_scores = cross_val_score(
    forest_reg, housing_prepared, housing_labels,
    scoring='neg_mean_squared_error', cv=10
)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores, "随机森林")

In [ ]:
# 保存模型
model_path = Path('my_model')
model_path.mkdir(exist_ok=True)
joblib.dump(forest_reg, model_path / 'random_forest_model.pkl')
print(f"模型已保存至: {model_path / 'random_forest_model.pkl'}")

## 7. 超参数调优

In [ ]:
# 网格搜索最优超参数
# 参数取10的幂次方是常用的搜索策略
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg_tuning = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(
    forest_reg_tuning, param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True,
    n_jobs=-1
)

grid_search.fit(housing_prepared, housing_labels)
print("最优参数:", grid_search.best_params_)

In [ ]:
# 查看所有参数组合的评估结果
cvres = grid_search.cv_results_
print("网格搜索结果:")
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(f"  RMSE: ${np.sqrt(-mean_score):,.2f}  参数: {params}")

In [ ]:
# 特征重要性分析
best_model = grid_search.best_estimator_
feature_importances = best_model.feature_importances_

# 构建特征名称列表
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.get_feature_names_out())
all_attribs = num_attribs + extra_attribs + cat_one_hot_attribs

# 按重要性排序
importance_ranking = sorted(
    zip(feature_importances, all_attribs),
    reverse=True
)

print("特征重要性排名:")
for importance, attr in importance_ranking:
    print(f"  {attr}: {importance:.4f}")

## 8. 测试集评估

In [ ]:
# 在测试集上评估最终模型
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

# 应用相同的预处理流水线
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(f"最终模型 - 测试集 RMSE: ${final_rmse:,.2f}")

In [ ]:
# 计算95%置信区间
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2

ci = np.sqrt(stats.t.interval(
    confidence,
    len(squared_errors) - 1,
    loc=squared_errors.mean(),
    scale=stats.sem(squared_errors)
))

print(f"测试集 RMSE 95% 置信区间: [${ci[0]:,.2f}, ${ci[1]:,.2f}]")

In [ ]:
# 预测结果分析
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 预测值 vs 真实值
axes[0].scatter(y_test, final_predictions, alpha=0.3)
axes[0].plot([0, 500000], [0, 500000], 'r--', label='理想预测线')
axes[0].set_xlabel('真实房价')
axes[0].set_ylabel('预测房价')
axes[0].set_title('预测值 vs 真实值')
axes[0].legend()

# 残差分布
residuals = final_predictions - y_test
axes[1].hist(residuals, bins=50, edgecolor='black')
axes[1].axvline(x=0, color='r', linestyle='--')
axes[1].set_xlabel('残差 (预测值 - 真实值)')
axes[1].set_ylabel('频数')
axes[1].set_title('残差分布')

plt.tight_layout()
plt.show()

## 9. 模型部署与监控

生产环境部署需考虑以下要点：

1. **模型版本管理**：使用版本控制跟踪模型文件和训练代码
2. **数据漂移监控**：定期检测输入数据分布变化
3. **性能退化预警**：设置 RMSE 阈值，触发模型重训练
4. **A/B 测试**：新模型上线前与现有模型对比
5. **回滚机制**：保留历史模型，支持快速回滚

In [ ]:
# 保存最终模型和预处理流水线
joblib.dump(final_model, model_path / 'final_model.pkl')
joblib.dump(full_pipeline, model_path / 'preprocessing_pipeline.pkl')

print("部署文件已生成:")
print(f"  - {model_path / 'final_model.pkl'}")
print(f"  - {model_path / 'preprocessing_pipeline.pkl'}")

In [ ]:
# 推理演示：加载模型并预测
loaded_model = joblib.load(model_path / 'final_model.pkl')
loaded_pipeline = joblib.load(model_path / 'preprocessing_pipeline.pkl')

# 模拟新数据
sample_data = X_test.iloc[:5]
sample_prepared = loaded_pipeline.transform(sample_data)
sample_predictions = loaded_model.predict(sample_prepared)

print("推理演示 - 预测结果:")
for i, (pred, actual) in enumerate(zip(sample_predictions, y_test.iloc[:5])):
    print(f"  样本{i+1}: 预测 ${pred:,.0f}, 实际 ${actual:,.0f}, 误差 ${abs(pred-actual):,.0f}")